In [126]:
# hide
# skip
! [ -e /content ] && pip install -Uqq fastai # upgrade fastai on colab

In [127]:
# default_exp collab
# default_class_lvl 3

In [128]:
# export
from fastai.tabular.all import *
from fastai.collab import *

In [129]:
# hide
from nbdev.showdoc import *

# Collaborative filtering
> Tools to add to the [fastai collab](https://docs.fast.ai/collab.html) to make the learning transferable

## Loading `users`/`items` embeddings from a pretrained model

In a collab model, to load a pretrained vocabulary, we need to adapt the embeddings of the  vocabulary used for the pre-training to the vocabulary of our current collab corpus.

In [171]:
# export 
def match_embeds(
    old_wgts:dict, # Embedding weights of the pretrained model
    old_vocab:list, # Vocabulary (tokens and labels) of the corpus used for pretraining
    new_vocab:dict # Current collab corpus vocabulary (`users` and `items`)
) -> dict:
    """
    Convert the `users` and `items` (possibly saved as `0.module.encoder.weight` and `1.attn.lbs_weight.weight` respectively) 
    embedding in `old_wgts` to go from `old_vocab` to `new_vocab`
    """
    import pdb; pdb.set_trace()
    u_bias, u_wgts = None, old_wgts.get('0.module.encoder.weight')
    print(f"{u_wgts.shape = }")
    i_bias, i_wgts = old_wgts.get('1.attn.lbs_bias.weight', None), old_wgts.get('1.attn.lbs_weight.weight')
    print(f"{i_wgts.shape = }")
    u_wgts_m, i_wgts_m = u_wgts.mean(0), i_wgts.mean(0)
    new_u_wgts = u_wgts.new_zeros((len(new_vocab['token']), u_wgts.size(1)))
    print(f"{new_u_wgts.shape = }")
    new_i_wgts = i_wgts.new_zeros((len(new_vocab['label']), i_wgts.size(1)))
    print(f"{new_i_wgts.shape = }")
    if u_bias is not None:
        u_bias_m = u_bias.mean(0)
        new_u_bias = u_bias.new_zeros((len(new_vocab['token']), 1))
    if i_bias is not None:
        i_bias_m = i_bias.mean(0)
        new_i_bias = i_bias.new_zeros((len(new_vocab['label']), 1))
    u_old = old_vocab[0]
    u_old_o2i = u_old.o2i if hasattr(u_old, 'o2i') else {w:i for i,w in enumerate(u_old)}
    i_old = old_vocab[1]
    i_old_o2i = i_old.o2i if hasattr(i_old, 'o2i') else {w:i for i,w in enumerate(i_old)}
    u_miss, i_miss = 0, 0
    for i,w in enumerate(new_vocab['token']):
        idx = u_old_o2i.get(w, -1)
        new_u_wgts[i] = u_wgts[idx] if idx>=0 else u_wgts_m
        if u_bias is not None: new_u_bias[i] = u_bias[idx] if idx>=0 else u_bias_m
        if idx == -1: u_miss = u_miss + 1
    for i,w in enumerate(new_vocab['label']):
        idx = i_old_o2i.get(w, -1)
        new_i_wgts[i] = i_wgts[idx] if idx>=0 else i_wgts_m
        if i_bias is not None: new_i_bias[i] = i_bias[idx] if idx>=0 else i_bias_m
        if idx == -1: i_miss = i_miss + 1
    old_wgts['0.module.encoder.weight'] = new_u_wgts
    if '0.module.encoder_dp.emb.weight' in old_wgts: old_wgts['0.module.encoder_dp.emb.weight'] = new_u_wgts.clone()
    if u_bias is not None: pass
    old_wgts['1.attn.lbs_weight.weight'] = new_i_wgts
    if '1.attn.lbs_weight_dp.emb.weight' in old_wgts: old_wgts['1.attn.lbs_weight_dp.emb.weight'] = new_i_wgts.clone()
    if i_bias is not None: old_wgts['1.attn.lbs_bias.weight'] = new_i_bias
    return old_wgts, u_miss, i_miss

## Create a `Learner`

In [163]:
# export
def load_pretrained_keys(
    model, # Model architecture
    wgts:dict # Model weights
) -> tuple:
    "Load relevant pretrained `wgts` in `model"
    sd = model.state_dict()
    u_wgts, u_bias = wgts.get('0.module.encoder.weight', None), None
    print(f"inside load_keys{u_wgts.shape = }")
    if u_wgts is not None: sd['u_weight.weight'].data = u_wgts.data
    if u_bias is not None: sd['u_bias.weight'].data = u_bias.data
    i_wgts, i_bias = wgts.get('1.attn.lbs_weight.weight', None), wgts.get('1.attn.lbs_bias.weight', None)
    print(f"inside load{i_wgts.shape = }")
    if i_wgts is not None: sd['i_weight.weight'].data = i_wgts.data
    if i_bias is not None: sd['i_bias.weight'].data = i_bias.data
    return model.load_state_dict(sd)

In [139]:
# export
class CollabLearner(Learner):
    "Basic class for a `Learner` in Collab."
    @delegates(save_model)
    def save(self, 
        file:str, # Filename for the state_directory of model
        **kwargs):
        """
        Save model and optimizer state (if `with_opt`) to `self.path/self.model_dir/file`
        Save `self.dls.classes` to `self.path.self.model_dir/collab_vocab.pkl`
        """
        model_file = join_path_file(file, self.path/self.model_dir, ext='.pth')
        vocab_file = join_path_file(file+'_vocab', self.path/self.model_dir, ext='.pkl')
        save_model(model_file, self.model, getattr(self,'opt', None), **kwargs)
        save_pickle(vocab_file, self.dls.classes)
        return model_file
    
    def load_vocab(self,
        wgts_fname:str, #Filename of the saved weights
        vocab_fname:str, # Saved vocabulary filename in pickle format
        model=None # Model to load parameters from, deafults to `Learner.model`
    ):
        "Load the vocabulary (`users` and/or `items`) from a pretrained model and adapt it to the collab vocabulary."
        old_vocab = load_pickle(vocab_fname)
        new_vocab = self.dls.classes
        distrib_barrier()
        wgts = torch.load(wgts_fname, map_location=lambda storage,loc: storage)
        if 'model' in wgts: wgts = wgts['model'] # Just in case the pretrained model was saved with an optimizer
        wgts, *_ = match_embeds(wgts, old_vocab, new_vocab)
        load_pretrained_keys(self.model if model is None else model, wgts)
        return self

In [133]:
show_doc(CollabLearner)

<h2 id="CollabLearner" class="doc_header"><code>class</code> <code>CollabLearner</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>CollabLearner</code>(**`dls`**, **`model`**, **`loss_func`**=*`None`*, **`opt_func`**=*`Adam`*, **`lr`**=*`0.001`*, **`splitter`**=*`trainable_params`*, **`cbs`**=*`None`*, **`metrics`**=*`None`*, **`path`**=*`None`*, **`model_dir`**=*`'models'`*, **`wd`**=*`None`*, **`wd_bn_bias`**=*`False`*, **`train_bn`**=*`True`*, **`moms`**=*`(0.95, 0.85, 0.95)`*) :: `Learner`

Basic class for a `Learner` in Collab.

It works exactly as a normal `learner`, the only difference is that it also saves the `items` vocabulary used by `self.model`

The following function lets us quickly create a `Learner` for collaborative filtering from the data.

In [134]:
# export
@delegates(Learner.__init__)
def collab_learner(dls, n_factors=50, use_nn=False, emb_szs=None, layers=None, config=None, y_range=None, loss_func=None, pretrained=False, **kwargs):
    "Create a Learner for collaborative filtering on `dls`."
    emb_szs = get_emb_sz(dls, ifnone(emb_szs, {}))
    if loss_func is None: loss_func = MSELossFlat()
    if config is None: config = tabular_config()
    if y_range is not None: config['y_range'] = y_range
    if layers is None: layers = [n_factors]
    if use_nn: model = EmbeddingNN(emb_szs=emb_szs, layers=layers, **config)
    else:      model = EmbeddingDotBias.from_classes(n_factors, dls.classes, y_range=y_range)
    learn = CollabLearner(dls, model, loss_func=loss_func, **kwargs)
    if pretrained:
        try: fnames = [list(learn.path.glob(f'**/clas/*clas*.{ext}'))[0] for ext in ['pth', 'pkl']] 
        except: IndexError: print(f'The model in {learn.path} is incomplete, re-train it'); raise
        learn = learn.load_vocab(*fnames, model=learn.model)
    return learn

If `use_nn=False`, the model used is an `EmbeddingDotBias` with `n_factors` and `y_range`. Otherwise, it's a `EmbeddingNN` for which you can pass `emb_szs` (will be inferred from the `dls` with `get_emb_sz` if you don't provide any), `layers` (defaults to `[n_factors]`) `y_range`, and a `config` that you can create with `tabular_config` to customize your model. 

`loss_func` will default to `MSELossFlat` and all the other arguments are passed to `Learner`.

In [135]:
path = untar_data(URLs.ML_SAMPLE)
ratings = pd.read_csv(path/'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,73,1097,4.0,1255504951
1,561,924,3.5,1172695223
2,157,260,3.5,1291598691
3,358,1210,5.0,957481884
4,130,316,2.0,1138999234


In [136]:
dls = CollabDataLoaders.from_df(ratings, bs=64)
dls.show_batch()

,userId,movieId,rating
0,212,1721,2.5
1,19,364,3.0
2,128,539,5.0
3,460,1270,4.0
4,134,500,4.0
5,665,1270,5.0
6,105,6539,3.0
7,475,3114,3.5
8,130,4886,3.0
9,243,380,3.0


In [137]:
with tempfile.TemporaryDirectory() as d:
    learn = collab_learner(dls, y_range=(0,5), path=d)
    learn.fit(1)
    
    # Test save created a file
    learn.save('tmp')
    assert (Path(d)/'models/tmp.pth').exists()
    assert (Path(d)/'models/tmp_vocab.pkl').exists()

epoch,train_loss,valid_loss,time
0,2.477255,2.338984,00:00


In [172]:
# hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_utils.ipynb.
Converted 01_layers.ipynb.
Converted 02_text.models.core.ipynb.
Converted 03_text.learner.ipynb.
Converted 04_metrics.ipynb.
Converted 05_collab.ipynb.
Converted index.ipynb.
